# Usecase II: Fake Reviews Detection
Ref: https://www.kaggle.com/code/crimsonred/fake-review-detection

In [31]:
# Download and unzip data
!wget http://data.insideairbnb.com/united-kingdom/england/london/2023-03-14/data/reviews.csv.gz -P ../data/
!gunzip -kf ../data/reviews.csv.gz

--2023-04-24 20:26:15--  http://data.insideairbnb.com/united-kingdom/england/london/2023-03-14/data/reviews.csv.gz
Resolving www-proxy.ao.ericsson.se (www-proxy.ao.ericsson.se)... 150.236.2.23
Connecting to www-proxy.ao.ericsson.se (www-proxy.ao.ericsson.se)|150.236.2.23|:8080... connected.
Proxy request sent, awaiting response... 200 OK
Length: 158880598 (152M) [application/x-gzip]
Saving to: ‘../data/reviews.csv.gz’

reviews.csv.gz      100%[===================>] 151.52M  1.57MB/s    in 2m 21s  

2023-04-24 20:28:38 (1.07 MB/s) - ‘../data/reviews.csv.gz’ saved [158880598/158880598]



In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')
nltktk.download('punkt')

In [1]:
from datetime import datetime, date
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer

default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english') # or any other list of your choice

%matplotlib inline

In [32]:
df = pd.read_csv('../data/reviews.csv')

In [34]:
df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,13913,80770,2010-08-18,177109,Michael,My girlfriend and I hadn't known Alina before ...
1,13913,367568,2011-07-11,19835707,Mathias,Alina was a really good host. The flat is clea...
2,13913,529579,2011-09-13,1110304,Kristin,Alina is an amazing host. She made me feel rig...
3,13913,595481,2011-10-03,1216358,Camilla,"Alina's place is so nice, the room is big and ..."
4,13913,612947,2011-10-09,490840,Jorik,"Nice location in Islington area, good for shor..."


In [38]:
df['reviewer_id'].value_counts()

227163707    133
100002221     79
74176596      77
7222993       75
67753141      66
            ... 
19813950       1
63248517       1
51532597       1
140763691      1
489030148      1
Name: reviewer_id, Length: 1145305, dtype: int64